# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
import random
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

class DecisionTree:
    def __init__(self, min_samples=1, max_depth=np.inf, max_features=np.inf):
        self.tree = None
        self.min_samples = min_samples
        self.max_depth = max_depth
        self.features_number = None
        self.node_splits_on_feature = []
        self.feature_importances_ = None 
        self.max_features = max_features

    # check the purity of data
    @staticmethod
    def pure(data):
        # take the last column of data
        label = data[:, -1]
        unique_classes = np.unique(label)
        return len(unique_classes) == 1

    @staticmethod
    def classify(data):
        """:returns the class which occurs most of all"""
        label = data[:, -1]
        unique_classes, unique_classes_counts = np.unique(label, return_counts=True)
        return unique_classes[unique_classes_counts.argmax()]

    @staticmethod
    def splits(data, max_features):
        """:returns all potential splits for given data"""
        potential_splits = {}
        _, columns = np.shape(data)
        columns = [column_index for column_index in range(columns - 1)]
        if len(columns) > max_features:
            columns = random.sample(columns, max_features)

        for column_index in columns:
            values = data[:, column_index]
            unique_values = np.unique(values)
            potential_splits[column_index] = unique_values
        return potential_splits

    @staticmethod
    def split_data(data, column_to_split, value_to_split):
        split_column = data[:, column_to_split]
        data_left = data[split_column <= value_to_split]
        data_right = data[split_column > value_to_split]
        return data_left, data_right

    # calculate entropy
    @staticmethod
    def entropy(data):
        label = data[:, -1]
        _, counts = np.unique(label, return_counts=True)
        probabilities = counts / counts.sum()
        entropy = sum(probabilities * -np.log2(probabilities))
        return entropy

    @staticmethod
    def overall_entropy(data_left, data_right):
        """calculate entropy for a single split"""
        data_points_len = len(data_left) + len(data_right)
        left_data_percent = len(data_left) / data_points_len
        right_data_percent = len(data_right) / data_points_len

        return (left_data_percent * DecisionTree.entropy(data_left)
                + right_data_percent * DecisionTree.entropy(data_right))

    # find best split
    @staticmethod
    def best_split(data, splits):
        """finds the best way to split data depending on each split entropy"""
        overall_entropy = np.inf
        for column in splits:
            for value in splits[column]:
                data_left, data_right = DecisionTree.split_data(data, column, value)
                curr_overall_entropy = DecisionTree.overall_entropy(data_left, data_right)
                if curr_overall_entropy <= overall_entropy:
                    overall_entropy = curr_overall_entropy
                    best_split_column = column
                    best_split_value = value
        return best_split_column, best_split_value

    def features_importance(self):
        all_nodes = len(self.node_splits_on_feature)
        node_counts = Counter(self.node_splits_on_feature)
        features_importance_ = {node: node_count / all_nodes for node, node_count in node_counts.most_common()}
        features_importance = []
        for i in range(self.features_number):
            if i in features_importance_.keys():
                features_importance.append(features_importance_[i])
            else:
                features_importance.append(0)
        self.feature_importances_ = features_importance

    def build_tree(self, data_frame, counter=0):
        """recursively builds a tree"""
        # prepare data
        if counter == 0:
            data = data_frame.values
        else:
            data = data_frame
        # when to stop
        if DecisionTree.pure(data) or len(data) < self.min_samples or counter >= self.max_depth:
            return DecisionTree.classify(data)
        else:
            counter += 1
            splits = DecisionTree.splits(data, self.max_features)
            split_column, split_value = DecisionTree.best_split(data, splits)
            data_left, data_right = DecisionTree.split_data(data, split_column, split_value)
            self.node_splits_on_feature.append(split_column)

            # check for empty nodes
            if len(data_left) == 0 or len(data_right) == 0:
                return DecisionTree.classify(data)
            # create subtree
            key = "column {} :split_by_value {}".format(split_column, split_value)
            sub_tree = {key: []}

            # recursion
            sub_tree[key].append(DecisionTree.build_tree(self, data_left, counter))
            sub_tree[key].append(DecisionTree.build_tree(self, data_right, counter))
            return sub_tree

    @staticmethod
    def classify_data(data, tree):
        """goes through the tree in order to find class of given exemplar"""
        node = list(tree.keys())[0]
        _, feature_name, _, split_value = node.split()
        feature_name = int(feature_name)
        if data[feature_name] <= float(split_value):
            sub_tree = tree[node][0]
        else:
            sub_tree = tree[node][1]
        if isinstance(sub_tree, dict):
            return DecisionTree.classify_data(data, sub_tree)
        else:
            return sub_tree

    def fit(self, train, target):
        data_frame = pd.DataFrame(train)
        _, self.features_number = np.shape(data_frame)
        data_frame['label'] = target
        self.tree = DecisionTree.build_tree(self, data_frame)

    def predict(self, data_frame):
        if not isinstance(data_frame, pd.DataFrame):
            data_frame = pd.DataFrame(data_frame)
        return data_frame.apply(self.classify_data, axis=1, args=(self.tree,))
        self.features_importance()
    
    def predict_proba(self, data_frame):
      if not isinstance(data_frame, pd.DataFrame):
            data_frame = pd.DataFrame(data_frame)
      prediction = data_frame.apply(self.classify_data, axis=1, args=(self.tree,))
      probability = []
      for class_ in prediction.values:
          class_prob = []
          for i in range(self.features_number-1):
            if i == class_:
              class_prob.append(1.)
            else:
              class_prob.append(0.)
          probability.append(class_prob)
      return probability

if __name__ == '__main__':
    # preparing data
    print(iris.data)
    iris = load_iris()
    train_data, test_data, train_target, test_target = train_test_split(iris.data, iris.target, train_size=0.7)

    # sklearn model
    sklearn_model = DecisionTreeClassifier()
    sklearn_model.fit(train_data, train_target)
    sklearn_prediction = sklearn_model.predict(test_data)

    # my model
    model = DecisionTree(min_samples=5, max_depth=5)
    model.fit(train_data, train_target)
    prediction = model.predict(test_data)

    print("sklearn tree realization", accuracy_score(sklearn_prediction, test_target))
    print("My tree realization", accuracy_score(prediction, test_target))


[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

**Защита**

In [ ]:
from sklearn.metrics import roc_auc_score
test_probabilities = sklearn_model.predict_proba(test_data)

roc_auc_value = roc_auc_score(test_target, test_probabilities, multi_class='ovr')
print("sklearn ROC-AUC на тестовой выборке:", roc_auc_value) 

mytest_probabilities = model.predict_proba(test_data)
my_roc_auc_value = roc_auc_score(test_target, mytest_probabilities, multi_class='ovr')
print("My tree realization ROC-AUC на тестовой выборке:", roc_auc_value) 

sklearn ROC-AUC на тестовой выборке: 0.9838383838383838
My tree realization ROC-AUC на тестовой выборке: 0.9838383838383838


**Построить Матрицу сопряженности**

In [ ]:
from sklearn.metrics import confusion_matrix

my_tree_confusion_matrix = confusion_matrix(test_target, prediction)
my_tree_confusion_matrix = pd.DataFrame(my_tree_confusion_matrix)
display(my_tree_confusion_matrix)

,0,1,2
0,16,2,0
1,0,14,1
2,0,0,12


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

In [ ]:
# загружаем данные
url = "https://raw.githubusercontent.com/mileniummi/data_for_ML/main/churn.csv?token=ANW6GMFOORSICXPOQU7QBWTBQMFGS"
churn = pd.read_csv(url)

In [ ]:
%%capture
!pip install category_encoders
from category_encoders import TargetEncoder

churn_target = churn["Exited"]
churn["Gender"] = TargetEncoder().fit_transform(churn["Gender"], churn_target)
churn["Geography"] = TargetEncoder().fit_transform(churn["Geography"], churn_target)
churn_data = churn.drop(["RowNumber", "CustomerId", 'Surname', "Exited"], axis = 1)

In [ ]:
train_data, test_data, train_target, test_target = train_test_split(churn_data, churn_target, train_size=0.7)

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

class RandomForest:
    def __init__(self, N=100, max_depth=100, min_samples=1):
        self.N = N
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.trees = []
    
    def get_params(self,deep=True):
        return {'N': self.N, 'max_depth': self.max_depth, 'min_samples': self.min_samples}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def fit(self, values, target):
        _, feature_number = np.shape(values)
        k = int(np.sqrt(feature_number))
        for i in range(self.N):
            model = DecisionTreeClassifier(max_features=k, max_depth=self.max_depth, min_samples_leaf=self.min_samples)
            state = random.randint(1, 10000)
            values_sample = values.sample(n=len(values), random_state = state, replace=True)
            target_sample = target.sample(n=len(values), random_state = state, replace=True)
            model.fit(values_sample, target_sample)
            self.trees.append(model)

    def predict(self, data):
        predictions = []
        for model in self.trees:
            predictions.append(model.predict(data))
        predictions = pd.DataFrame(data=predictions)
        return round(predictions.mean())

    def features_importance(self):
        predictions = []
        for model in self.trees:
            predictions.append(model.feature_importances_)
        predictions = pd.DataFrame(data=predictions)
        return predictions.mean()

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'N': [10, 100, 300, 600],'max_depth': [None,1, 5, 8, 15],'min_samples': [1, 2, 5, 10]}
forest_grid = GridSearchCV(RandomForest(), params, scoring='accuracy')

In [ ]:
%time
forest_grid.fit(train_data,train_target)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 10 µs


GridSearchCV(cv=None, error_score=nan,
             estimator=<__main__.RandomForest object at 0x7f2d384a9790>,
             iid='deprecated', n_jobs=None,
             param_grid={'N': [10, 100, 300, 600],
                         'max_depth': [None, 1, 5, 8, 15],
                         'min_samples': [1, 2, 5, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [ ]:
forest_grid.score(test_data, test_target) 

0.8733333333333333

In [ ]:
best_params = forest_grid.best_params_
best_params

{'N': 100, 'max_depth': 15, 'min_samples': 5}

In [ ]:
model = RandomForest(100, 15, 5)
model.fit(train_data,train_target)
prediction = model.predict(test_data)
print(accuracy_score(prediction, test_target))

0.8563333333333333


**Построить матрицу сопряженности**

In [ ]:
from sklearn.metrics import confusion_matrix

my_tree_confusion_matrix = confusion_matrix(test_target, prediction)
my_tree_confusion_matrix = pd.DataFrame(my_tree_confusion_matrix)
display(my_tree_confusion_matrix)

,0,1
0,2321,88
1,343,248


In [ ]:
display(pd.DataFrame(index=churn_data.columns, data=model.features_importance().values, columns=['importance']).sort_values(by='importance', ascending=False))

,importance
Age,0.292726
NumOfProducts,0.176503
Balance,0.126886
EstimatedSalary,0.107217
CreditScore,0.104322
Tenure,0.058143
IsActiveMember,0.055662
Geography,0.046661
Gender,0.018886
HasCrCard,0.012993
